In [1]:
# Chicago API
import os
import requests
from bs4 import BeautifulSoup
import lxml
import pandas as pd
def getBusStop(df, spectag):
    tagdf = df.find_all(spectag)
    returndf = []
    for tag in tagdf:
        temp = tag.text.strip() 
        returndf.append(temp) 
    returndf = pd.Series(returndf)
    return returndf

routesurl = "http://www.ctabustracker.com/bustime/api/v3/getroutes?key=kUXKRKMhBsjB3a32q5UcLZ4YX"
routes = requests.get(url=routesurl)
routesdata = BeautifulSoup(routes.text,"xml")
routetag = routesdata.find_all("rt")
routenumbers = []
for tag in routetag: 
    temp = tag.text.strip() 
    routenumbers.append(temp) 
routenumbers = pd.Series(routenumbers) 
templateurl = "https://www.ctabustracker.com/bustime/api/v3/getdirections?key=kUXKRKMhBsjB3a32q5UcLZ4YX&rt="
templateurl2 = "https://www.ctabustracker.com/bustime/api/v3/getstops?key=kUXKRKMhBsjB3a32q5UcLZ4YX&rt="
templateurl3 = "&dir="
stopsloc = pd.DataFrame(columns=["StopId","Lat","Lon"])
for i in routenumbers:
    directionurl = "".join([templateurl, i])
    direction = requests.get(url=directionurl)
    directiondata = BeautifulSoup(direction.text,"xml")
    directiontag = directiondata.find_all("id")
    dir = []
    for tag in directiontag: 
        temp = tag.text.strip() 
        dir.append(temp) 
    dir = pd.Series(dir) 
    stopurl = "".join([templateurl2, i])
    stopurl = "".join([stopurl, templateurl3])
    for w in dir:
        stopurltemp = "".join([stopurl, w])
        stops = requests.get(url=stopurltemp)
        stopsdata = BeautifulSoup(stops.text, "xml")
        StopId = getBusStop(stopsdata, "stpid")
        Lat = getBusStop(stopsdata, "lat")
        Lon = getBusStop(stopsdata, "lon")
        stops_df = pd.DataFrame({'StopId': StopId, "Lat": Lat, "Lon": Lon})
        stopsloc = pd.concat([stopsloc, stops_df], axis =0)


In [2]:
stopsloc = stopsloc.iloc[:,[0,2,1]]
stopsloc

,StopId,Lon,Lat
0,1564,-87.623207000001,41.840606999999
1,12713,-87.639435000001,41.879288000001
2,17046,-87.631083,41.879457
3,1108,-87.636101000002,41.879177000001
4,4732,-87.628093000001,41.879491
...,...,...,...
81,14258,-87.685911999999,42.030722
82,18322,-87.685016000001,42.023165999999
83,14254,-87.685354999999,42.026477
84,14256,-87.685657000001,42.028933


In [3]:
import os
import pandas as pd

dalbus = pd.read_csv("DALBus.csv")

dalbus = dalbus[["stop_id","WKT"]]
dalbus_cord = dalbus.WKT.str.split(" ",expand=True)
dalbus_cord.columns={"Longitude","Point","Latitude"}
dalbus_cord.Longitude = dalbus_cord.Longitude.replace(r"\(","",regex=True)
dalbus_cord.Latitude = dalbus_cord.Latitude.replace(r"\)","",regex=True)
dalbus = dalbus.join(dalbus_cord, how="outer")
dalbus = dalbus[["stop_id", "Longitude","Latitude"]]

In [4]:
import os
os.getcwd()
ChicagoStops = stopsloc
DallasStops = dalbus
DallasStops.columns=["StopId","Lon","Lat"]
DCStops = pd.read_csv("Metro_Bus_Stops.csv")
DCStops.columns=["StopId","Lon","Lat"]
LAStops = pd.read_csv("stops.txt")
LAStops = LAStops[["stop_id","stop_lat","stop_lon"]]
LAStops.columns={"StopId","Lon","Lat"}
ChicagoStops = ChicagoStops[["StopId","Lon","Lat"]]
ChicagoStops["City"] = "Chicago"
DallasStops["City"] = "Dallas"
DCStops["City"] = "DC"
LAStops["City"] = "Los Angeles"
BusStops = pd.concat([ChicagoStops,LAStops,DCStops, DallasStops],axis=0)


In [11]:
import pandas as pd
BusStops = BusStops.reset_index(drop=True)
BusStops.to_csv("BusStops.csv")
BusStops

,StopId,Lon,Lat,City
0,1564,-87.623207000001,41.840606999999,Chicago
1,12713,-87.639435000001,41.879288000001,Chicago
2,17046,-87.631083,41.879457,Chicago
3,1108,-87.636101000002,41.879177000001,Chicago
4,4732,-87.628093000001,41.879491,Chicago
...,...,...,...,...
37839,33338,-96.792761,32.784013,Dallas
37840,33357,-96.986743,32.930609,Dallas
37841,33373,-96.808074,32.794071,Dallas
37842,33383,-96.746982,32.909406,Dallas


In [8]:
DALRail = pd.read_csv("DALRail.csv")
DALRail.columns=["StationId","Lon","Lat"]
DCRail = pd.read_csv("Metro_Stations_in_DC.csv")
DCRail.columns=["StationId","Lon","Lat"]
LARail = pd.read_csv("la_stops.txt")
LARail = LARail[["stop_id","stop_lat","stop_lon"]]
LARail.columns={"StationId","Lon","Lat"}
ChicagoRail = pd.read_csv("CTA_RailStations.csv")
DALRail["City"] = "Dallas"
ChicagoRail["City"] = "Chicago"
DCRail["City"] = "DC"
LARail["City"] = "Los Angeles"
RailStations = pd.concat([ChicagoRail,LARail,DCRail, DALRail],axis=0)

In [13]:
RailStations = RailStations.reset_index(drop=True)
RailStations.to_csv("RailStations.csv")
RailStations

,StationId,Lon,Lat,City
0,580.0,-87.756698,41.851781,Chicago
1,1680.0,-87.747227,42.026252,Chicago
2,150.0,-87.724333,41.853865,Chicago
3,1020.0,-87.708547,41.929736,Chicago
4,550.0,-87.729183,41.952948,Chicago
...,...,...,...,...
684,32934.0,-96.82284,32.758891,Dallas
685,32984.0,-96.82405,32.753575,Dallas
686,32985.0,-96.824305,32.748985,Dallas
687,33086.0,-96.788202,32.666780,Dallas
